# La théorie des jeux

Auteur : Philippe Mathieu, CRISTAL Lab, SMAC Team, University of Lille, email : philippe.mathieu@univ-lille.fr

Contributeurs : Louisa Fodil (CRISTAL/SMAC), Céline Petitpré (CRISTAL/SMAC)

Creation : 18/01/2018


## Introduction

La théorie des jeux est la discipline qui étudie de manière formelle les interactions que peuvent avoir plusieurs individus dans une situation conflictuelle. Le terme "jeu" n'a a priori rien de ludique, Von Neumann l'ayant par exemple développée pour la modélisation des relations Franco-Américaines de 1945 et notamment lors de l'épisode de la baie des cochons. La théorie des jeux décompose les jeux en fonction du nombre de joueurs (généralement 2), puis principalement en deux grandes familles : les jeux simultanés, dans lesquels les joueurs jouent simultanément et donc ne savent pas en jouant ce que joue l'adversaire (papier-feuiile-ciseaux par exemple) et les jeux non simultanés où chacun joue tour à tour (les échecs par exemple). 

Nous nous plaçons ici dans le cadre des **jeux à deux joueurs, simultanés**. On représente en général ce type de jeu par une matrice de gains. Les coups d'un des deux joueurs se trouvent en abscisse et ceux de l'autre en ordonnée. L'intersection de chaque case caractérise une situation de jeu. On y note les points qui seront distribués à chacun des deux joueurs. Pour information, la théorie des jeux décompose encore ces jeux en deux autres familles, les jeux à somme nulle dans lesquels tout ce que perd l'un est gagné par l'autre, et les jeux à somme non nulle dans lesquels les points distribués sont différents selon les coups joués.

Cette feuille a pour objectif de montrer comment représenter et générer des jeux, puis comment calculer les différents équilibres d'un jeu.

On trouvera plus de détails sur [Wikipedia](http://www.wikidedia.org)



# Une matrice de gains

Tout d'abord, il est necessaire de pouvoir coder une matrice de gains. Une classe `Game` va nous
permettre de stocker les gains des deux joueurs pour une situation donnée. 
Un objet Game prend un tableau de couples en paramètre, correspondant aux scores de chaque issue, ainsi que le tableau des noms des actions correspondantes.
En interne, Le package *numpy* offre un objet `Array` idéal pour stocker et manipuler cela. Pour un jeu à `n` coups, nous créons un Array `n*n` de couples de valeurs `(x,y)` avec `x` le gain du joueur 1 et `y` le gain du joueur 2. 

In [54]:
import numpy as np
import math

class Game:
    def __init__(self, tab, actions):
        self.actions=actions
        m=np.array(tab,dtype=[('x', object), ('y', object)])
        self.size = int(math.sqrt(len(tab)))
        self.scores=m.reshape(self.size,self.size)

In [55]:
# Exemples de jeu
dp =[(3,3),(0,5),(5,0),(1,1)]   # Dilemme du prisonnier : 1 equilibrium
gs=[(3,2),(1,1),(0,0),(2,3)]     # Guerre des sexes : 2 equilibria
mp=[(1,-1),(-1,1),(-1,1),(1,-1)] # Matching pennies : 0 equilibrium
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)] # papier feuille ciseaux  : 0 equilibrium
g = Game(dp,['C','D'])
g.scores

array([[(3, 3), (0, 5)],
       [(5, 0), (1, 1)]], dtype=[('x', 'O'), ('y', 'O')])

# La notion d'équilibre

On explique ici l'hypothèse de rationalité des agents en économie, et la recherche d'équilibre.

Les économistes se sont très vite emparés de ces modèles pour étudier différents problèmes économiques. Afin d'identifier la meilleure manière de jouer à un jeu fixé, ils se sont basés sur la notion abstraite de rationalité du joueur. Partant du principe que, dans le pire des cas, l'adversaire est aussi rationnel, le raisonnement que je m'applique doit donc aussi être appliqué à mon adversaire. Très vite, plusieurs définitions de la rationalité sont apparues, définissant ainsi un (ou éventuellement plusieurs) point fixe d'un jeu, point vers lequel rationnellement, si tout le monde réfléchis de manière identique, on parviendra :
1. *L'équilibre de Nash*, du nom de son inventeur, John Nash. L'équilibre de Nash est une situation de non regret de chacun des joueurs. *"nous avons joué simultanément, mais maintenant que je sais ce que l'autre a joué, je n'ai aucun regret"*
2. *L'optimum de Pareto*, du nom de son inventeur, Wilfried Pareto. l'optimum de Pareto est une situation où aucune situation n'est supérieure pour les deux joueurs, et donc ils n'ont aucun interêt collectivement à changer.
3. *L'équilibre en stratégies dominantes*. Un joueur "rationnel" ne jouera jamais une de ses stratégies si elle dominée par une autre de ses stratégies. On peut donc "simplifier" un jeu en éliminant itérativement les stratégies dominées.



## L'équilibre de Nash

Pour calculer la situation de "non regret" correspondant à l'équilibre de Nash il suffit de noter les réponses du joueur 1 les plus adaptées à chaque stratégie du joueur 2 (donc calculer les max(x) dans la chaque colonne), puis calculer les meilleures réponses du joueur 2 aux stratégies du joueur 1 (donc calculer les max(y) dans chaque ligne). Si une issue possede 2 max, c'est un équilibre de Nash. Selon les jeux, il peut bien sûr y en avoir 1, plusieurs ou pas du tout. 
L'usage d'un `np.Array` facilite énormément les choses puisqu'il est possible d'avoir les vecteurs de valeurs max en ligne ou en colonne. Il suffit donc de fabriquer des matrices booléennes dans chaque cas et d'en faire un `ou` logique.


In [56]:
def nash(self):
    max_x = np.matrix(self.scores['x'].max(0)).repeat(self.size, axis=0)
    bool_x = self.scores['x'] == max_x
    max_y = np.matrix(self.scores['y'].max(1)).transpose().repeat(self.size, axis=1)
    bool_y = self.scores['y'] == max_y
    bool_x_y = bool_x & bool_y
    return self.scores[bool_x_y]

#### Prenons le jeu de la guerre des sexes 
Principe : Un couple s'est donné rendez-vous pour la soirée, mais aucun ne parvient à se souvenir si c'est pour assister à un match de foot ou aller à l'opéra. 
Le mari préférerait aller voir le foot, la femme aimerait aller à l'opéra. 
Tous deux préfèrent cependant aller au même endroit plutôt que d'être seuls.
Si l'homme va au stade et que sa femme s'y trouve, il obtient 3.
Si il est seul au stade, il obtient 1.
Si il va à l'opéra et que sa femme est au stade, il obtient 0.
Si il va à l'opéra et que sa femme est aussi à l'opéra, il obtient 2.
(Même chose pour la femme en inversant opéra et stade)
On obtient donc la matrice gs suivante, puis on teste l'équilibre de Nash.

In [57]:
gs=[(3,2),(1,1),(0,0),(2,3)]
g = Game(gs,['Opera','Stade'])
print(nash(g))

[(3, 2) (2, 3)]


On remarque ici que le jeu possède deux équilibres de Nash qui correspondent aux situations où le couple est ensemble.

#### Prenons maintenant le jeu très connu de "Papier Feuille Ciseaux" et testons s'il possède un ou plusieurs équilibres de Nash

In [58]:
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])
print(nash(g))

[]


On remarque ici que ce jeu ne possède pas d'équilibre de Nash

## L'optimum de Pareto



In [59]:
def isPareto(self, t, s):
    return True if (len(s)==0) else (s[0][0]<=t[0] or s[0][1]<=t[1]) and isPareto(self,t, s[1:])

def pareto(self):
        res = list()
        liste = self.scores.flatten()
        for s in liste:
            if(isPareto(self,s,liste)):res.append(tuple(s))
        return res

#### Reprenons le jeu de la guerre des sexes

In [60]:
gs=[(3,2),(1,1),(0,0),(2,3)]
g = Game(gs,['Opera','Stade'])
print(pareto(g))

[(3, 2), (2, 3)]


On remarque qu'il y a deux optima de Pareto pour la guerre des sexes qui sont les mêmes que pour les équilibres de Nash.

#### Tester maintenant l'optimum de Pareto avec le jeu "Pierre Feuille Ciseaux" 

In [19]:
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])

## L'équilibre en stratégie dominante

Un équilibre en stratégies dominantes est toujours un équilibre de Nash mais l'inverse n'est pas vrai. Il existe deux types de stratégie dominantes : "strictement dominante" (toujours >)  et "faiblement dominante" (>=). Si on a éliminé les stratégies *strictement* dominées et qu'il ne reste plus qu'une stratégie pour chaque joueur, alors c'est aussi le seul équilibre de Nash. Si on a éliminé les stratégies *faiblement* dominées et qu'il ne reste qu'une stratégie pour chaque joueur, alors c'est un équilibre de Nash, mais il peut y en avoir d'autres (et selon l'ordre d'élimination on tombe sur l'un ou l'autre)

In [67]:
# CELINE  idealement il faut une méthode qui fournit la matrice simplifiée
# On passe une matrice ; ça renvoie une matrice (éventuellement avec 1 seule case ou eventuellement la même)

def dominantStrategy(self, strict="True"):
        lignesDominees = []
        colonnesDominees = []
        findDominated = True
        while (findDominated and (len(lignesDominees) != self.size - 1) and (len(colonnesDominees) != self.size - 1)):
            findDominated = False
            #on regarde les lignes dominées
            for i in range(self.size-1) :
                ligne1 = self.scores['x'][i]
                ligne2 = self.scores['x'][i+1]
                if compare(self, ligne1, ligne2, colonnesDominees, strict):
                    lignesDominees += [i]
                    findDominated = True
                if compare(self, ligne2, ligne1, colonnesDominees, strict):
                    ligneDominees += [i+1]
                    findDominated = True
            #on regarde les colonnes dominées
            for i in range(self.size-1) :
                c1 = self.scores['y'].transpose()[i]
                c2 = self.scores['y'].transpose()[i+1]
                if compare(self, c1, c2, lignesDominees, strict):
                    colonnesDominees += [i]
                    findDominated = True
                if compare(self, c2, c1, lignesDominees, strict):
                    colonnesDominees += [i+1]
                    findDominated = True
        #return lignesDominees, colonnesDominees
        return result(self, lignesDominees, colonnesDominees)
    
def compare(self, l1,l2, tab, strict):
    dominated = True
    for i in range(self.size) :
        if (strict) :
            if ((l1[i] < l2[i] and i not in tab) or i in tab):
                dominated = dominated and True
            else :
                dominated = dominated and False
        else  :
            if ((l1[i] <= l2[i] and i not in tab) or i in tab):
                dominated = dominated and True
            else :
                dominated = dominated and False
    return dominated   

def result(self, lignesDominees, colonnesDominees ):
    resMatrix = self.scores
    for i in range (self.size):
            if i in lignesDominees : 
                resMatrix['x'][i] = None
            if i in colonnesDominees :
                resMatrix['y'].transpose()[i] = None
    print(resMatrix)
    
    if (len(lignesDominees) == len(colonnesDominees) == self.size - 1):
        for i in range (self.size):
            if i not in lignesDominees : 
                resX = i
            if i not in colonnesDominees :
                resY = i
        #return self.scores[resX][resY]
        return self.actions[resX], self.actions[resY]
    else :
        return "Il n'existe pas de strategie dominante"
    

#### Prenons le jeu de la guerre des sexes. 
On teste les stratégies dominées strictement et faiblement, si une stratégie est dominée, elle sera indiquée comme "None"

In [71]:
gs=[(3,2),(1,1),(0,0),(2,3)]     
g = Game(gs,['Opera','Foot'])

print("Avec l'élimination des stratégies strictement dominées : ")
print(dominantStrategy(g))
print(" ")

print("Avec l'élimination des stratégies faiblement dominées : ")
print(dominantStrategy(g, strict="False"))

Avec l'élimination des stratégies strictement dominées : 
[[(3, 2) (1, 1)]
 [(0, 0) (2, 3)]]
Il n'existe pas de strategie dominante
 
Avec l'élimination des stratégies faiblement dominées : 
[[(3, 2) (1, 1)]
 [(0, 0) (2, 3)]]
Il n'existe pas de strategie dominante


## Toutes ces méthodes peuvent bien sur être intégrées dans la classe Game.

On a donc une classe Game avec un cxonstructeur à 2 paramètres : la matrice de gains, et le vecteur de stratégies accessibles
puis 3 méthodes
nash() qui renvoie ??
pareto() qui renvoie ???
dominantSimplification() qui renvoie la matrice simplifiée

CELINE : y'a pas un moyen de décrire une classe et ses methodes en Markdown ?

CELINE : Faire le import qui va bien puis oon attaque avec les exemples qui suivent. Le bloc ci dessous est destiné à disparaitre de cette feuille, mais par contre, on "l'importe"

In [77]:
import numpy as np
import math

class Game:
    def __init__(self, tab, actions):
        self.actions=actions
        m=np.array(tab,dtype=[('x', object), ('y', object)])
        self.size = int(math.sqrt(len(tab)))
        self.scores=m.reshape(self.size,self.size)

    def nash(self):
        max_x = np.matrix(self.scores['x'].max(0)).repeat(self.size, axis=0)
        bool_x = self.scores['x'] == max_x
        max_y = np.matrix(self.scores['y'].max(1)).transpose().repeat(self.size, axis=1)
        bool_y = self.scores['y'] == max_y
        bool_x_y = bool_x & bool_y
        return self.scores[bool_x_y]
    
    def isPareto(self, t, s):
        return True if (len(s)==0) else (s[0][0]<=t[0] or s[0][1]<=t[1]) and self.isPareto(t, s[1:])
    
    def pareto(self):
        res = list()
        liste = self.scores.flatten()
        for s in liste:
            if(self.isPareto(s,liste)):res.append(tuple(s))
        return res
    
  
    def dominantStrategy(self, strict="True"):
        lignesDominees = []
        colonnesDominees = []
        findDominated = True
        while (findDominated and (len(lignesDominees) != self.size - 1) and (len(colonnesDominees) != self.size - 1)):
            findDominated = False
            #on regarde les lignes dominées
            for i in range(self.size-1) :
                ligne1 = self.scores['x'][i]
                ligne2 = self.scores['x'][i+1]
                if self.compare( ligne1, ligne2, colonnesDominees, strict):
                    lignesDominees += [i]
                    findDominated = True
                if self.compare(ligne2, ligne1, colonnesDominees, strict):
                    ligneDominees += [i+1]
                    findDominated = True
            #on regarde les colonnes dominées
            for i in range(self.size-1) :
                c1 = self.scores['y'].transpose()[i]
                c2 = self.scores['y'].transpose()[i+1]
                if self.compare( c1, c2, lignesDominees, strict):
                    colonnesDominees += [i]
                    findDominated = True
                if self.compare(c2, c1, lignesDominees, strict):
                    colonnesDominees += [i+1]
                    findDominated = True
        #return lignesDominees, colonnesDominees
        return self.result(lignesDominees, colonnesDominees)
    
    def compare(self, l1,l2, tab, strict):
        dominated = True
        for i in range(self.size) :
            if (strict) :
                if ((l1[i] < l2[i] and i not in tab) or i in tab):
                    dominated = dominated and True
                else :
                    dominated = dominated and False
            else  :
                if ((l1[i] <= l2[i] and i not in tab) or i in tab):
                    dominated = dominated and True
                else :
                    dominated = dominated and False
        return dominated   
    
    def result(self, lignesDominees, colonnesDominees ):
        resMatrix = np.array(self.scores, copy=True) 
        for i in range (self.size):
            if i in lignesDominees : 
                resMatrix['x'][i] = None
            if i in colonnesDominees :
                resMatrix['y'].transpose()[i] = None
        print(resMatrix)
        if (len(lignesDominees) == len(colonnesDominees) == self.size - 1):
            for i in range (self.size):
                if i not in lignesDominees : 
                    resX = i
                if i not in colonnesDominees :
                    resY = i
            #return self.scores[resX][resY]
            return self.actions[resX], self.actions[resY]
        else :
            return "Il n'existe pas de strategie dominante"

### Le dilemme du prisonnier

Le Dilemme du prisonnier, identifié par M. Flood and M. Dresher de la Rand Corporation en 1950, est un modèle de théorie des jeux spécialement créé pour montrer que l'équilibre de Nash n'est pas toujours une bonne idée.


#### Testons maintenant les équilibres de Nash, Pareto et les éliminations de stratégies dominées pour le dilemme du prisonnier.


In [78]:
g = Game(dp,['C','D'])


print(g.nash())
print("Il existe un équilibres de Nash pour le dilemme du prisonner")
print(" ")

print(g.pareto())
print("Il n'existe pas d'optimum de Pareto pour le dilemme du prisonnier")
print(" ")

print(g.dominantStrategy())
print("La stratégie strictement dominante pour le dilemme du prisonnier est la stratégie où les deux joueurs choisissent de trahir.")
print(" ")

print(g.dominantStrategy(strict="False"))
print("Une stratégie faiblement dominante pour le dilemme du prisonnier est la stratégie où les deux jouers choisissent de trahir.")



[(1, 1)]
Il existe un équilibres de Nash pour le dilemme du prisonner
 
[(3, 3), (0, 5), (5, 0)]
Il n'existe pas d'optimum de Pareto pour le dilemme du prisonnier
 
[[(None, None) (None, 5)]
 [(5, None) (1, 1)]]
('D', 'D')
La stratégie strictement dominante pour le dilemme du prisonnier est la stratégie où les deux joueurs choisissent de trahir.
 
[[(None, None) (None, 5)]
 [(5, None) (1, 1)]]
('D', 'D')
Une stratégie faiblement dominante pour le dilemme du prisonnier est la stratégie où les deux jouers choisissent de trahir.


## Amusons nous

### Créer une matrice de jeu au hasard

In [81]:
x = np.random.randint(0, 5, (3,3))
y = np.random.randint(0, 5, (3,3))
couples = [(a,b) for a,b in zip(x.flatten(),y.flatten())]
g = Game(couples, None)
print(g.scores)
print(g.nash())

[[(3, 0) (0, 0) (3, 1)]
 [(2, 0) (2, 0) (3, 0)]
 [(1, 0) (3, 4) (4, 2)]]
[(3, 4)]


### Générer toutes les matrices

En Python il est facile d'énumérer tous les jeux à 2 coups à partir de valeurs fixées. Par exemple tous jeux que l'on peut construire avec les valeurs 1 et 2.
La Librairie `itertools` fournit de nombreux itérateurs efficaces, notamment pour les combinaisons, permutations et produits cartésiens. Ici c'est le produit cartésien des valeurs qui nous interesse.
On peut ensuite par exemple compter combien de jeux ont 0,1 ou plusieurs équilibres de Nash.


In [82]:
import itertools;
import random;

def numberOfGames(valeurs, nbCoups):
    return len(valeurs)**((nbCoups**2)*2)

print(numberOfGames([1,2],2))

def enumAllGames(valeurs, nbCoups):
    res = [q for q in itertools.product([p for p in itertools.product(list(valeurs), repeat=2)], repeat=nbCoups**2)]
    return [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]

n = enumAllGames([1,2],2)
print("Impression de 10 jeux aléatoires trouvés sur "+str(numberOfGames([1,2],2)))
for i in range (10):
    print(random.choice(n))
        
def countNashEquilibria(valeurs, coups):
    results = [Game(i,None).nash().size for i in enumAllGames(valeurs, coups)]
    return dict((i,results.count(i)) for i in set(results))

# Combien de jeux à 2 coups batis sur (1,2) on x equilbres de Nash
countNashEquilibria([1,2],2)

256
Impression de 10 jeux aléatoires trouvés sur 256
[(2, 2), (2, 2), (2, 1), (2, 1)]
[(1, 1), (2, 1), (2, 1), (1, 1)]
[(1, 1), (1, 1), (1, 1), (2, 1)]
[(2, 1), (2, 1), (1, 2), (2, 2)]
[(1, 1), (1, 1), (2, 2), (2, 2)]
[(2, 1), (1, 2), (2, 2), (2, 2)]
[(1, 2), (2, 2), (1, 1), (1, 1)]
[(2, 2), (2, 1), (1, 2), (1, 2)]
[(2, 1), (1, 1), (2, 2), (1, 1)]
[(2, 2), (2, 1), (2, 1), (2, 1)]


{0: 2, 1: 44, 2: 114, 3: 80, 4: 16}

# Trouver des jeux avec des contraintes particulières
Cherchons par exemple, les jeux à deux coups dont les valeurs sont prises dans (1,2) et qui ont exactement les mêmes equilibres nash et pareto

In [83]:
nbCoups=2
res = [q for q in itertools.combinations([p for p in itertools.product([0,1,3,5], repeat=2)], nbCoups**2)]
games = [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]
print(str(len(games))+" jeux étudiés.")
r = []
for g in games:
    if ((sorted(Game(g,None).pareto()) == sorted(Game(g,None).nash().tolist())) and (len(set(g)) == len(g))):
        r.append(g)
r


1820 jeux étudiés.


[[(0, 0), (0, 1), (0, 3), (1, 3)],
 [(0, 0), (0, 1), (0, 3), (1, 5)],
 [(0, 0), (0, 1), (0, 3), (3, 3)],
 [(0, 0), (0, 1), (0, 3), (3, 5)],
 [(0, 0), (0, 1), (0, 3), (5, 3)],
 [(0, 0), (0, 1), (0, 3), (5, 5)],
 [(0, 0), (0, 1), (0, 5), (1, 5)],
 [(0, 0), (0, 1), (0, 5), (3, 5)],
 [(0, 0), (0, 1), (0, 5), (5, 5)],
 [(0, 0), (0, 1), (1, 0), (3, 3)],
 [(0, 0), (0, 1), (1, 0), (3, 5)],
 [(0, 0), (0, 1), (1, 0), (5, 3)],
 [(0, 0), (0, 1), (1, 0), (5, 5)],
 [(0, 0), (0, 1), (1, 1), (3, 3)],
 [(0, 0), (0, 1), (1, 1), (3, 5)],
 [(0, 0), (0, 1), (1, 1), (5, 3)],
 [(0, 0), (0, 1), (1, 1), (5, 5)],
 [(0, 0), (0, 1), (1, 3), (3, 3)],
 [(0, 0), (0, 1), (1, 3), (3, 5)],
 [(0, 0), (0, 1), (1, 3), (5, 3)],
 [(0, 0), (0, 1), (1, 3), (5, 5)],
 [(0, 0), (0, 1), (1, 5), (3, 5)],
 [(0, 0), (0, 1), (1, 5), (5, 5)],
 [(0, 0), (0, 1), (3, 0), (5, 3)],
 [(0, 0), (0, 1), (3, 0), (5, 5)],
 [(0, 0), (0, 1), (3, 1), (5, 3)],
 [(0, 0), (0, 1), (3, 1), (5, 5)],
 [(0, 0), (0, 1), (3, 3), (5, 3)],
 [(0, 0), (0, 1), (3

# Bibliographie

A refaire avec uniquelment de la théorie des jeux

- Von Neumann et Morgenstern etc ....